# Problem Set 4

## Question 2

In this exercise, we numerically study the behaviors of the wild and the pairs bootstrap by considering the following model:
\begin{align}\label{Q4}\tag{1}
Y_i=Z_{1i} + Z_{2i} + Z_{3i} + \varrho_0 Z_{1i} Z_{2i} + (1+\lambda_0 Z_{1i})\epsilon_i~,
\end{align}
for $i=1,\ldots, n$. The sample $\{Y_i,Z_{1i},Z_{2i},Z_{3i}\}_{i=1}^n$ is generated as follows.

(1) Generate $V_i,Z_{2i}$ and $Z_{3i}$ from the standard normal distribution with equal correlation $0.2$, and then set
    \begin{align}
    Z_{1i} = \frac{\exp(V_i)-E[\exp(V_i)]}{\sqrt{\mathrm{Var}(\exp(V_i))}}~,
    \end{align}
    where $E[\exp(V_i)]=\exp(0.5)$ and $\sqrt{\mathrm{Var}(\exp(V_i))}=2.16$ (approximately).

(2) Independently, $\epsilon_i$ is generated from a mixture of a $N(-1/9,1)$ variable with probability $0.9$ and a $N(1,4)$ variable with probability $0.1$. When coding, you may first draw a sample $\eta_i$ from a Bernoulli distribution with success probability $0.9$, and then you draw $\epsilon_i$ from $N(-1/9,1)$ if $\eta_i=1$ and otherwise draw $\epsilon_i$ from $N(1,4)$. There may be a Python package for implementing this in a compact fashion, but, if not, I would just set $\epsilon_i=\eta_i W_{1i} + (1-\eta_i)W_{2i}$ for $W_{1i}\sim N(-1/9,1)$ and independently $W_{2i}\sim N(1,4)$.

(3) Generate $Y_i$ according to (1) for each pair $(\varrho_0,\lambda_0)$ to be specified.

Given $\{Y_i,Z_{1i},Z_{2i},Z_{3i}\}_{i=1}^n$, we then consider the following regression:
  \begin{align}
  Y_i=\alpha_0+\beta_1Z_{1i} + \beta_2 Z_{2i} + \beta_3 Z_{3i} + U_i~.
  \end{align}
Note that if $\varrho_0=0$, then $E[U_i|Z_{1i},Z_{2i},Z_{3i}]=0$ (the model is correctly specified), and if $\varrho_0\neq 0$, then $E[U_i|Z_{1i},Z_{2i},Z_{3i}]\neq 0$ so the model is misspecified (but one still has $E[U_i(1,Z_{1i},Z_{2i},Z_{3i})]=0$). The model exhibits conditional heteroskedasticity if $\lambda_0\neq 0$. In what follows, set $\lambda_0=1$ and $n=200$.

 You should manually generate the bootstrap samples and compute the bootstrap statistics, as described on p.51 of the lecture notes.

(a) Let $\varrho_0=0$. Compute the empirical rejection rates of the two-sided $t$-test for $\mathrm H_0: \beta_1=1$ at the significance level $\alpha=5\%$, based on the standard normal approximation, the pairs bootstrap and the wild bootstrap using Radamacher weights. Use $5000$ Monte Carlo replications and $200$ bootstrap repetitions. The closer the empirical rejection rates are to $\alpha$ (the nominal rate), the better the test performs. See the pdf of the problem set for a sketch of the coding structure.

In [1]:
import numpy as np
import statsmodels.api as sm
from sympy.stats import Rademacher, sample
from tqdm import tqdm

#Settings in question
sample_size = 200
mu = [0,0,0]
var_cov_mat = [[1,0.2,0.2],[0.2,1,0.2],[0.2,0.2,1]]

rho_0 = 0
ld_0 = 1

#Data generating process
def dgp(sample_size, mu, var_cov_mat, rho_0, ld_0):
    #For v, z2, z3
    v, z2, z3 = np.random.multivariate_normal(mu, var_cov_mat, sample_size).T
    
    #For z1
    E_expv = np.exp(0.5)
    sd_expv = 2.16
    z1 = (np.exp(v) - E_expv)/sd_expv

    #Step 2
    eta = np.random.binomial(n = 1, p = 0.9, size = sample_size)
    w1 = np.random.normal(-1/9, 1, sample_size)
    w2 = np.random.normal(1, 2, sample_size)
    epsilon = eta*w1 + (1 - eta)*w2

    #step 3 
    y = z1 + z2 + z3 + rho_0*z1*z2 + (1+ld_0*z1)*epsilon
    return y, z1, z2, z3

In [2]:
#Settings in question
R = 5000                  #Number of Monte Carlo replications
b_sample_size = 200       #Sample size in bootstrapping
B = 200                   #Bootstrap reptitions
b_0 = 1                   #Null hypothesis value of beta1

#Start to compute the empirical rejection rule
def emp_rej_rule(sample_size, mu, var_cov_mat, rho_0, ld_0, R, b_sample_size, B, b_0):
    
    RejN_list = []
    RejP_list = []
    RejW_list = []
    
    for rep in tqdm(range(0,R)):
        
        #Generate a sample
        s = dgp(sample_size, mu, var_cov_mat, rho_0, ld_0)
        y = s[0]
        x_woc = np.transpose(s[1:])

        #Compute t-stat t_n after doing OLS regression
        x = sm.add_constant(x_woc)
        model = sm.OLS(y, x)
        model = model.fit(cov_type='HC1')
        t_n = (model.params[1] - b_0)/(model.bse[1]) #just get it for beta1
        t_n = np.abs(t_n)
        
        #Constrained least square for wild bootstrap
        x_cls_woc = np.transpose(s[2:])
        x_cls = sm.add_constant(x_cls_woc)
        y_cls = y-(s[1]*b_0)
        model_cls = sm.OLS(y_cls, x_cls)
        model_cls = model_cls.fit(cov_type='HC1')
        cls_resid = model_cls.resid

        #Get ready for bootstrap
        t_n_P_list = []
        t_n_w_list = []
        
        #Bootrapping starts
        for b in range(0, B):
            
            #pair bootstrap
            rd = np.random.choice(sample_size, b_sample_size)  #draw (with replacement) `b_sample_size` number from range(0, sample_size)
            BootP_y = y[rd]
            BootP_x_woc = x_woc[rd,]
            BootP_x = sm.add_constant(BootP_x_woc)
            model_P = sm.OLS(BootP_y, BootP_x)
            model_P = model_P.fit(cov_type='HC1')
            t_n_P = (model_P.params[1] - model.params[1])/(model_P.bse[1]) #just get it for beta1
            t_n_P_list.append(np.abs(t_n_P))
            
            #wild bootstrap #Redimicher
            X = Rademacher('X')
            r = sample(X, size = b_sample_size)
            Bootw_u = cls_resid*r
            Bootw_y = model_cls.predict() + s[1]*b_0  + Bootw_u
            model_w = sm.OLS(Bootw_y, x)
            model_w = model_w.fit(cov_type='HC1')
            t_n_w = (model_w.params[1] - b_0)/(model_w.bse[1]) #just get it for beta1, cls estimator of beta1 = 1
            t_n_w_list.append(np.abs(t_n_w))

        CriticalP = np.percentile(t_n_P_list, 95)
        CriticalW = np.percentile(t_n_w_list, 95)

        RejN_list.append(1*(t_n>1.96))
        RejP_list.append(1*(t_n>CriticalP))
        RejW_list.append(1*(t_n>CriticalW))

    RejN = np.mean(RejN_list)
    RejP = np.mean(RejP_list)
    RejW = np.mean(RejW_list)
    
    print("Empirical Rejection rates based on:")
    print("1. The standard normal approximation: " + str(RejN))
    print("2. The pair bootstrap: " + str(RejP))
    print("3. The wild bootstrap using Radamacher weight: " + str(RejW))

In [3]:
#Run the bootstrap
emp_rej_rule(sample_size, mu, var_cov_mat, rho_0, ld_0, R, b_sample_size, B, b_0)

100%|██████████| 5000/5000 [41:04<00:00,  2.03it/s]

Empirical Rejection rates based on:
1. The standard normal approximation: 0.1666
2. The pair bootstrap: 0.0712
3. The wild bootstrap using Radamacher weight: 0.0584


(b) Repeat part (a) by setting $\varrho_0\in\{-0.5,0.5\}$ (so the model is misspecified), but now consider the two-sided $t$ test for $\mathrm H_0: \beta_1=1+\varrho_0 b$ for $b=0.41$ (note that $1+\varrho_0 b$ is the slope parameter for $Z_{1i}$ in the best linear prediction of $Y_i$ given $1,Z_{1i},Z_{2i}$, and $Z_{3i}$).

In [4]:
#For rho_0 = -0.5

#Update rho_0 and b_0
rho_0 = -0.5
b_0 = 1+rho_0*0.41

#Run the bootstrap
emp_rej_rule(sample_size, mu, var_cov_mat, rho_0, ld_0, R, b_sample_size, B, b_0)

100%|██████████| 5000/5000 [40:41<00:00,  2.05it/s]

Empirical Rejection rates based on:
1. The standard normal approximation: 0.1714
2. The pair bootstrap: 0.0616
3. The wild bootstrap using Radamacher weight: 0.0538


In [5]:
#For rho_0 = 0.5

#Update rho_0 and b_0
rho_0 = 0.5
b_0 = 1+rho_0*0.41 

#Run the bootstrap
emp_rej_rule(sample_size, mu, var_cov_mat, rho_0, ld_0, R, b_sample_size, B, b_0)

100%|██████████| 5000/5000 [40:44<00:00,  2.05it/s] 

Empirical Rejection rates based on:
1. The standard normal approximation: 0.1806
2. The pair bootstrap: 0.0746
3. The wild bootstrap using Radamacher weight: 0.0628
